In [84]:
import os
import warnings
import sys

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import tree, preprocessing, metrics, model_selection
import mlflow
import mlflow.sklearn

import logging

logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

SEED = 10
np.random.seed(SEED)

Carregamento dos Dados

In [4]:
df_kb = pd.read_csv('../Data/kobe_dataset.csv')
target_col = 'shot_made_flag'


#label_map = df_wine[['target', 'target_label']].drop_duplicates()
#drop_cols = ['target_label']
#df_wine.drop(drop_cols, axis=1, inplace=True)
#print(df_kb.shape)

#df_kb.head()
#df_kb.keys()

In [5]:
len(df_kb)

30697

In [3]:
#!mlflow server --backend-store-uri sqlite:///mlruns.db --default-artifact-root ./artifacts --host 127.0.0.1

In [6]:
import matplotlib.pyplot as plt
import pandas
import numpy as np
import pycaret.classification as pc
%matplotlib inline
from pycaret.utils import version
cross_validation = True
fold_strategy = 'stratifiedkfold',
fold = 10
# Para usar o sqlite como repositorio
mlflow.set_tracking_uri("sqlite:///mlruns.db")

experiment_name = 'Kobe_Bryant_Shot_Experiment'
experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment is None:
    experiment_id = mlflow.create_experiment(experiment_name)
    experiment = mlflow.get_experiment(experiment_id)
experiment_id = experiment.experiment_id

#with mlflow.start_run(experiment_id=experiment_id, run_name = 'Pycaret Setup', nested=True):
configurator = pc.setup(data = df_kb, # Configurações de dados
             target = target_col
            )
#mlflow.end_run()

,Description,Value
0,session_id,7854
1,Target,shot_made_flag
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(30697, 25)"
5,Missing Values,True
6,Numeric Features,9
7,Categorical Features,14
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=['shot_id'],
                                      ml_usecase='classification',
                                      numerical_features=[],
                                      target='shot_made_flag',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=N...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', 'passthrough'),
                ('c

In [16]:
import matplotlib.pyplot as plt
import pandas
import numpy as np
import pycaret.classification as pc
%matplotlib inline
from pycaret.utils import version
cross_validation = True
fold_strategy = 'stratifiedkfold',
fold = 10
# Para usar o sqlite como repositorio
mlflow.set_tracking_uri("sqlite:///mlruns.db")

experiment_name = 'Kobe_Bryant_Shot_TestImplementation'
experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment is None:
    experiment_id = mlflow.create_experiment(experiment_name)
    experiment = mlflow.get_experiment(experiment_id)
experiment_id = experiment.experiment_id

#with mlflow.start_run(experiment_id=experiment_id, run_name = 'Pycaret Setup', nested=True):
#configurator = pc.setup(data = df_kb, # Configurações de dados
 #            target = target_col,
  #          preprocess = False
   #         )
#mlflow.end_run()

In [8]:
df_kb.isnull().sum()

action_type              0
combined_shot_type       0
game_event_id            0
game_id                  0
lat                      0
loc_x                    0
loc_y                    0
lon                      0
minutes_remaining        0
period                   0
playoffs                 0
season                   0
seconds_remaining        0
shot_distance            0
shot_made_flag        5000
shot_type                0
shot_zone_area           0
shot_zone_basic          0
shot_zone_range          0
team_id                  0
team_name                0
game_date                0
matchup                  0
opponent                 0
shot_id                  0
dtype: int64

In [9]:
len(df_kb)

30697

In [13]:

nulos = df_kb_all.isnull().sum()
df_kb = df_kb_all[df_kb_all['shot_made_flag'].notnull()].reset_index()
df_kb.isnull().sum()

index                 0
action_type           0
combined_shot_type    0
game_event_id         0
game_id               0
lat                   0
loc_x                 0
loc_y                 0
lon                   0
minutes_remaining     0
period                0
playoffs              0
season                0
seconds_remaining     0
shot_distance         0
shot_made_flag        0
shot_type             0
shot_zone_area        0
shot_zone_basic       0
shot_zone_range       0
team_id               0
team_name             0
game_date             0
matchup               0
opponent              0
shot_id               0
dtype: int64

5000

In [14]:
len(df_kb)

25697

In [90]:
# COLOCAR RUN DE LEITURA DE DADOS
# PARAMETROS: top_features,
# METRICS: SHAPE de cada base de dados
# ARTIFACTS: nenhum


top_features = ['lat','lon', 'minutes_remaining' , 'period', 'playoffs', 'shot_distance']
target_col = 'shot_made_flag'
target_col_label = 'shot_made_label'

with mlflow.start_run(experiment_id=experiment_id, run_name = 'PreparacaoDados', nested=True):
    
    #Leitura de dados
    path_kb_data_input= '../Data/kobe_dataset.csv'
    df_kb_all = pd.read_csv(path_kb_data_input)    
    mlflow.log_metric("Tamanho/Linhas - Base Entrada", df_kb_all.shape[0])
    
    #Descrição Variável alvo
    mapa ={0 : 'Errou', 1 : 'Cesta'}
    df_kb_all['shot_made_label'] = pd.DataFrame(df_kb_all [target_col].map(mapa))
    df_kb_all[[target_col, target_col_label]]
    
    
    #Remoção de dados Faltantes na Shot_made_Flag
    mlflow.log_metric("Quantidade de {} Faltante".format(target_col), df_kb_all['shot_made_flag'].isnull().sum())
    df_kb = df_kb_all[df_kb_all['shot_made_flag'].notnull()].reset_index()
    mlflow.log_metric("Tamanho/Linhas - Base sem dados faltantes", df_kb.shape[0])
    
    
    #Seleção de Features
    df_kb_tf = df_kb [top_features + ['shot_type', target_col]].copy()
    mlflow.log_param("top_features", top_features)
    
    #Filtro 2PT Field Goal
    
    df_kb_2PT = df_kb_tf[df_kb_tf['shot_type'] == '2PT Field Goal'].copy().drop('shot_type', axis=1)
    df_kb_2PT.to_parquet('../Data/Processed/data_filtered.parquet')
    
    
    # Separação da base com 80%/20% test_size=0.2
    #stratifyarray-like, default=None If not None, data is split in a stratified fashion, using this as the class labels.
    #shuffle = True
    df_kb_tt, df_kb_operation, ytrain, ytest = model_selection.train_test_split(df_kb_2PT, 
                                                                            df_kb_2PT[target_col],
                                                                            test_size=0.2,
                                                                            shuffle=True)
    
    df_kb_tt[target_col]      = ytrain
    df_kb_operation[target_col] = ytest
    
    
    mlflow.log_metric("Tamanho/Linhas - Base Treino/Teste", df_kb_tt.shape[0])
    mlflow.log_metric("Tamanho/Linhas - Base Operação", df_kb_operation.shape[0])
    
    #Base  3PT Field Goal
    df_kb_novelty = df_kb[df_kb['shot_type'] == '3PT Field Goal'].copy().drop('shot_type', axis=1)
    mlflow.log_metric("Tamanho/Linhas - Base Novidade", df_kb_novelty.shape[0]) 
    
    #Envio datasets para "/Data/operalization/base_{train|test}.parquet
    df_kb_tt.to_parquet('../Data/Operalization/base_train_test.parquet')
    df_kb_operation.to_parquet('../Data/Operalization/base_operation.parquet')
    df_kb_novelty.to_parquet('../Data/Operalization/base_novelty.parquet')
    
#label_map = df_wine[['target', 'target_label']].drop_duplicates()
#drop_cols = ['target_label']
#df_wine.drop(drop_cols, axis=1, inplace=True)
#print(df_kb.shape)

#df_kb.head()
#df_kb.keys()
    
    
mlflow.end_run()

In [58]:
target_col_label = 'shot_made_label'
path_kb_data_input= '../Data/kobe_dataset.csv'
df_kb_all = pd.read_csv(path_kb_data_input)
mapa ={0 : 'Errou', 1 : 'Cesta'}
df_kb_all['shot_made_label'] = pd.DataFrame(df_kb_all [target_col].map(mapa))
df_kb_label_map = df_kb_all[[target_col, target_col_label]].drop_duplicates()
df_kb_label_map

drop_cols = [target_col_label]
#df_kb_1 = df_kb_all.drop(drop_cols, axis=1)

#df_kb = df_kb[top_features + ['type', target_col]].copy()

KeyError: "['type', 'lng', 'minutes remaining'] not in index"

In [60]:
target_col_label = 'shot_made_label'
path_kb_data_input= '../Data/kobe_dataset.csv'
df_kb_all = pd.read_csv(path_kb_data_input)
df_kb_all.shape[0]

30697

In [61]:
len(df_kb_all)

30697

In [62]:
df_kb_all.shape[1]

25

In [72]:
df_kb_all.keys()

Index(['action_type', 'combined_shot_type', 'game_event_id', 'game_id', 'lat',
       'loc_x', 'loc_y', 'lon', 'minutes_remaining', 'period', 'playoffs',
       'season', 'seconds_remaining', 'shot_distance', 'shot_made_flag',
       'shot_type', 'shot_zone_area', 'shot_zone_basic', 'shot_zone_range',
       'team_id', 'team_name', 'game_date', 'matchup', 'opponent', 'shot_id',
       'shot_made_label'],
      dtype='object')

In [73]:
df_kb_all.head()

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,...,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date,matchup,opponent,shot_id,shot_made_label
0,Jump Shot,Jump Shot,10,20000012,33.9723,167,72,-118.1028,10,1,...,Right Side(R),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,1,NaN
1,Jump Shot,Jump Shot,12,20000012,34.0443,-157,0,-118.4268,10,1,...,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,2,Errou
2,Jump Shot,Jump Shot,35,20000012,33.9093,-101,135,-118.3708,7,1,...,Left Side Center(LC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,3,Cesta
3,Jump Shot,Jump Shot,43,20000012,33.8693,138,175,-118.1318,6,1,...,Right Side Center(RC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,4,Errou
4,Driving Dunk Shot,Dunk,155,20000012,34.0443,0,0,-118.2698,6,2,...,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,5,Cesta


In [77]:
'lng' in df_kb_all

False

In [78]:
'minutes remaining' in df_kb_all

False

In [85]:
df_kb_all ['shot_type'].drop_duplicates()

0     2PT Field Goal
10    3PT Field Goal
Name: shot_type, dtype: object

In [99]:
pc.get_metrics().Name.values

array(['Accuracy', 'AUC', 'Recall', 'Precision', 'F1', 'Kappa', 'MCC',
       'LogLoss'], dtype=object)

In [100]:
from sklearn.metrics import log_loss
pc.add_metric('logloss', 'LogLoss', log_loss, greater_is_better=False)

ValueError: id already present in metrics dataframe.

In [101]:
pc.get_metrics()

,Name,Display Name,Score Function,Scorer,Target,Args,Greater is Better,Multiclass,Custom
ID,,,,,,,,,
acc,Accuracy,Accuracy,<function accuracy_score at 0x000001E6DF7A0378>,accuracy,pred,{},True,True,False
auc,AUC,AUC,<function roc_auc_score at 0x000001E6DF793C80>,"make_scorer(roc_auc_score, needs_proba=True, e...",pred_proba,"{'average': 'weighted', 'multi_class': 'ovr'}",True,True,False
recall,Recall,Recall,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(recall_score, average=macro)",pred,{'average': 'macro'},True,True,False
precision,Precision,Prec.,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(precision_score, average=weighted)",pred,{'average': 'weighted'},True,True,False
f1,F1,F1,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(f1_score, average=weighted)",pred,{'average': 'weighted'},True,True,False
kappa,Kappa,Kappa,<function cohen_kappa_score at 0x000001E6DF7A0...,make_scorer(cohen_kappa_score),pred,{},True,True,False
mcc,MCC,MCC,<function matthews_corrcoef at 0x000001E6DF7A0...,make_scorer(matthews_corrcoef),pred,{},True,True,False
logloss,LogLoss,LogLoss,<function log_loss at 0x000001E6E8CD9378>,"make_scorer(log_loss, greater_is_better=False)",pred,{},False,True,True


In [103]:
s= pc.setup(data = df_kb_tt, 
                 target = target_col,
                 train_size=0.7,
                 silent = True,
                 fold_strategy = 'stratifiedkfold',
                 fold = fold,
                 log_experiment = True, 
                 experiment_name = experiment_name, 
                 log_plots = True
                )


,Description,Value
0,session_id,1236
1,Target,shot_made_flag
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(16228, 7)"
5,Missing Values,False
6,Numeric Features,3
7,Categorical Features,3
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:Logging experiment in MLFlow
INFO:logs:SubProcess save_model() called ==================================
INFO:logs:Initializing save_model()
INFO:logs:save_model(kwargs={}, verbose=False, prep_pipe_=Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=False, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[],
                                      target='shot_made_flag',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                nume...
                ('scaling', 'passthrough

In [104]:
pc.get_metrics()

,Name,Display Name,Score Function,Scorer,Target,Args,Greater is Better,Multiclass,Custom
ID,,,,,,,,,
acc,Accuracy,Accuracy,<function accuracy_score at 0x000001E6E8BD9598>,accuracy,pred,{},True,True,False
auc,AUC,AUC,<function roc_auc_score at 0x000001E6E8CDFEA0>,"make_scorer(roc_auc_score, needs_proba=True, e...",pred_proba,"{'average': 'weighted', 'multi_class': 'ovr'}",True,True,False
recall,Recall,Recall,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(recall_score, average=macro)",pred,{'average': 'macro'},True,True,False
precision,Precision,Prec.,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(precision_score, average=weighted)",pred,{'average': 'weighted'},True,True,False
f1,F1,F1,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(f1_score, average=weighted)",pred,{'average': 'weighted'},True,True,False
kappa,Kappa,Kappa,<function cohen_kappa_score at 0x000001E6E8BD9...,make_scorer(cohen_kappa_score),pred,{},True,True,False
mcc,MCC,MCC,<function matthews_corrcoef at 0x000001E6E8CD9...,make_scorer(matthews_corrcoef),pred,{},True,True,False


In [105]:
pc.add_metric('logloss', 'LogLoss', log_loss, greater_is_better=False)

Name                                                        LogLoss
Display Name                                                LogLoss
Score Function            <function log_loss at 0x000001E6E8CD9378>
Scorer               make_scorer(log_loss, greater_is_better=False)
Target                                                         pred
Args                                                             {}
Greater is Better                                             False
Multiclass                                                     True
Custom                                                         True
Name: logloss, dtype: object

In [108]:
bestmodel = pc.create_model('lr',
                                cross_validation = cross_validation, 
                                probability_threshold=0.5)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,LogLoss
Fold,,,,,,,,
0,0.5757,0.5952,0.4526,0.5767,0.5072,0.1441,0.1474,14.6548
1,0.5907,0.6142,0.4964,0.5900,0.5391,0.1759,0.1780,14.1379
2,0.5405,0.5520,0.4653,0.5269,0.4942,0.0762,0.0767,15.8710
3,0.5581,0.5734,0.4680,0.5482,0.5049,0.1103,0.1115,15.2629
4,0.6136,0.6253,0.5119,0.6195,0.5606,0.2212,0.2244,13.3474
5,0.5889,0.5982,0.5356,0.5791,0.5565,0.1745,0.1749,14.1987
6,0.5687,0.5952,0.4936,0.5590,0.5243,0.1325,0.1334,14.8980
7,0.5713,0.5933,0.5082,0.5605,0.5331,0.1386,0.1391,14.8068
8,0.5845,0.6161,0.4991,0.5796,0.5363,0.1637,0.1652,14.3508


INFO:logs:create_model_container: 1
INFO:logs:master_model_container: 1
INFO:logs:display_container: 2
INFO:logs:CustomProbabilityThresholdClassifier(C=1.0, class_weight=None,
                                     classifier=LogisticRegression(C=1.0,
                                                                   class_weight=None,
                                                                   dual=False,
                                                                   fit_intercept=True,
                                                                   intercept_scaling=1,
                                                                   l1_ratio=None,
                                                                   max_iter=1000,
                                                                   multi_class='auto',
                                                                   n_jobs=None,
                                                                   penalty='l2'

In [113]:
metr=pc.get_metrics()

In [119]:
metr

,Name,Display Name,Score Function,Scorer,Target,Args,Greater is Better,Multiclass,Custom
ID,,,,,,,,,
acc,Accuracy,Accuracy,<function accuracy_score at 0x000001E6E8BD9598>,accuracy,pred,{},True,True,False
auc,AUC,AUC,<function roc_auc_score at 0x000001E6E8CDFEA0>,"make_scorer(roc_auc_score, needs_proba=True, e...",pred_proba,"{'average': 'weighted', 'multi_class': 'ovr'}",True,True,False
recall,Recall,Recall,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(recall_score, average=macro)",pred,{'average': 'macro'},True,True,False
precision,Precision,Prec.,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(precision_score, average=weighted)",pred,{'average': 'weighted'},True,True,False
f1,F1,F1,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(f1_score, average=weighted)",pred,{'average': 'weighted'},True,True,False
kappa,Kappa,Kappa,<function cohen_kappa_score at 0x000001E6E8BD9...,make_scorer(cohen_kappa_score),pred,{},True,True,False
mcc,MCC,MCC,<function matthews_corrcoef at 0x000001E6E8CD9...,make_scorer(matthews_corrcoef),pred,{},True,True,False
logloss,LogLoss,LogLoss,<function log_loss at 0x000001E6E8CD9378>,"make_scorer(log_loss, greater_is_better=False)",pred,{},False,True,True


In [ ]:
metr= metrics.classification_report(df_kb_op[target_col], df_kb_op['operation_label'])
print(metr)

In [ ]:
from sklearn.metrics import log_loss
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
ll = log_loss(df_kb_op[target_col], df_kb_op['operation_label'])
f1 = f1_score(df_kb_op[target_col], df_kb_op['operation_label'])
acc= accuracy_score(df_kb_op[target_col], df_kb_op['operation_label'])
print("Log Loss:\t{:0.2f}\nF1-Score:\t{:0.2f}\nAcurácia:\t{:0.2f}".format(ll,f1*100,acc*100))